<a href="https://colab.research.google.com/github/andrew-veriga/Tensorflow-labs/blob/master/Course_1_Part_4_Lesson_2_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# продолжаем Hello World, пример компьютерного зрения
В предыдущем упражнении вы увидели, как создать нейронную сеть, которая выяснила проблему, которую вы пытались решить. Она показала пример явного обученного поведения. Конечно, в том случае это было немного излишним, потому что было бы проще написать функцию Y = 2x-1 напрямую, вместо того, чтобы пытаться использовать машинное обучение для изучения отношений между X и Y для фиксированного набора значений и расширения его на все значения.

Но как насчет сценария, в котором написание подобных правил намного сложнее - например, проблемы с компьютерным зрением? Давайте посмотрим на сценарий, в котором мы можем распознать различные предметы одежды, обученные из набора данных, содержащего 10 различных типов.

## Начало программирования

Начнем с нашего импорта TensorFlow

In [ ]:
import tensorflow as tf
print(tf.__version__)

Данные Fashion MNIST доступны непосредственно в API наборов данных tf.keras. Вы загружаете их как здесь:

In [ ]:
mnist = tf.keras.datasets.fashion_mnist

Вызов load_data для этого объекта даст вам два набора из двух списков, это будут обучающие и тестовые данные картинок, содержащих предметы одежды и их метки.

In [ ]:
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

Как выглядят эти значения? Давайте выведем картинку и ее метку из обучающего набора, чтобы посмотреть.
Поэкспериментируйте с различными индексами в массиве. Например, взгляните также на индекс 42


In [ ]:
import matplotlib.pyplot as plt
plt.imshow(training_images[0])
print(training_labels[0])
print(training_images[0])

Вы заметите, что все значения в числе находятся в диапазоне от 0 до 255. Если мы обучаем нейронную сеть, по разным причинам будет проще, если мы будем рассматривать все значения в диапазоне от 0 до 1, известный нам процесс **нормализация**. И, благодаря векторизации, в Python такой список легко нормализовать без циклов. Вы делаете это так:

In [ ]:
training_images  = training_images / 255.0
test_images = test_images / 255.0

Теперь вам может быть интересно, почему тут 2 комплекта: обучение и тестирование - помните, мы говорили об этом во вступлении?
Идея состоит в том, чтобы иметь 1 набор данных для обучения, а затем еще один набор данных, которого модель еще не видела ... чтобы увидеть, насколько хороша она будет в классификации значений. В конце концов, когда вы закончите, вы захотите попробовать это с данными, которые раньше не видели!

Давайте теперь соберем модель. Здесь довольно много новых понятий, но не волнуйтесь, вы их освоите.

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

**Sequential** - определяет ПОСЛЕДОВАТЕЛЬНОСТЬ слоев в нейронной сети

**Flatten** - Помните, наши изображения были квадратными, когда вы их распечатывали? Flatten просто берет этот квадрат и превращает его в одномерный вектор.

**Dense** - Добавляет слой нейронов

Каждому слою нейронов требуется  **activation function**, чтобы сообщить им, что делать. Есть много вариантов, но сейчас просто используйте эти.

**Relu** Фактически означает «если X>0 - возвращает X, иначе возвращает 0» - всё, что  делает эта функция активации - только передает те значения, которые больше нуля на следующий уровень в сети.

**Softmax** - берет набор значений и фактически выбирает самое большое, поэтому, например, если выходные данные последнего слоя выглядят как [0,1, 0,1, 0,05, 0,1, 9,5, 0,1, 0,05, 0,05, 0,05], то после Softmax результат будет таким: [0,0,0,0,1,0,0,0,0] - цель - сэкономить много кода!

Теперь, когда модель определена, следующее, что нужно сделать - создать ее. Вы делаете это, компилируя ее с указанием оптимизатора и функции потерь, как и раньше, а затем тренируете посредством метода **model.fit**, который сопоставит ваши тренировочные данные меткам обучения - то есть, выявит взаимосвязи между обучающими данными и их фактическими метками. И в будущем, если у вас есть данные, которые похожи на обучающие данные, то модель может их классифицировать.

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(training_images, training_labels, epochs=5)

Как только вы закончите тренировку - вы увидите значение точности в конце финальной эпохи. Она может выглядеть примерно как 0,9098. Это говорит о том, что ваша нейронная сеть на 91% точна в классификации тренировочных данных. То есть, она выяснила  шаблон совпадения между изображением и надписями, который работает в 91% случаев. Не здорово, но неплохо, учитывая, что она была тренирована только 5 эпох и закончила довольно быстро.
Но как она будет работать с новыми данными? Вот для чего у нас есть тестовые изображения. Мы можем вызвать метод **model.evaluate**, с двумя приготовленными тестовыми наборами, и он сообщит нам об ошибке для каждой пары вектор-метка. Давайте попробуем:


In [ ]:
model.evaluate(test_images, test_labels)

Мне  показало точность около 0,8838, что означает около 88%. Как и ожидалось, модель, вероятно, не будет так хороша с *незнакомыми* данными, как она была с данными, на которых была обучена! Проходя этот курс, вы будете искать способы улучшить ее.

Чтобы узнать больше, попробуйте следующие упражнения:

# Исследовательские упражнения

###Упражнение 1:
Запустите приведенный ниже код: он классифицирует каждое изображение из тестового набора, а затем печатает первую запись. Вывод метода **model.predict** - это список чисел. Как вы думаете, почему это так, и что представляют собой эти цифры?

In [ ]:
classifications = model.predict(test_images)

print(classifications[0])

Подсказка: попробуйте вывести значение (test_labels [0]) - и вы получите 9. Это поможет вам понять, почему этот список выглядит так, как он выглядит.

In [ ]:
print(test_labels[0])

На следующие вопросы необходимо ответить в лекции "Знакомство с компьютерным зрением - > Исследовательские упражнения: Модель распознавания изображений одежды"
###  Вопрос 1. Что представляет собой этот список?




### Вопрос 2. Как вы узнали из этого списка, что предмет - ботинок?





##Упражнение 2:
Давайте теперь посмотрим на слои в вашей модели. Экспериментируйте с различными значениями для слоя с 512 нейронами. Какие разные результаты вы получаете для Loss, времени тренировки и т.д.? Как вы думаете, почему?



In [ ]:
import tensorflow as tf
print(tf.__version__)


(training_images, training_labels) ,  (test_images, test_labels) = mnist.load_data()

training_images = training_images/255.0
test_images = test_images/255.0

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy')

model.fit(training_images, training_labels, epochs=5)

model.evaluate(test_images, test_labels)

classifications = model.predict(test_images)

print(classifications[0])
print(test_labels[0])

###Вопрос 3. Увеличьте количество нейронов в слое до 1024. Как это повлияет на время обучения и точность?


##Упражнение 3:

Что произойдет, если вы удалите слой Flatten()? Почему вы думаете, что это так?

Вы получаете ошибку о форме данных. Это подтверждает практическое правило, что первый слой в вашей сети должен иметь ту же форму, что и ваши данные. Изначально наши данные представляют собой изображения 28x28, и 28 слоев из 28 нейронов были бы неосуществимы, поэтому имеет смысл «расплющить» эти квадраты 28 х 28 в строки 784x1. Вместо того, чтобы писать код для такой обработки, мы добавляем слой Flatten() в начале, и когда потом массивы загружаются в модель, они автоматически вытягиваются для нас в вектор.


In [ ]:
import tensorflow as tf
print(tf.__version__)



(training_images, training_labels) ,  (test_images, test_labels) = mnist.load_data()

training_images = training_images/255.0
test_images = test_images/255.0

model = tf.keras.models.Sequential([#tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(64, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy')

model.fit(training_images, training_labels, epochs=5)

model.evaluate(test_images, test_labels)

classifications = model.predict(test_images)

print(classifications[0])
print(test_labels[0])

##Упражнение 4:
Рассмотрим финальные (выходные) слои. Почему их 10? Что произойдет, если бы у вас было количество, отличное от 10? Например, попробуйте тренировать сеть с 5

Вы получаете сообщение об ошибке, как только оно находит неожиданное значение. Еще одно практическое правило - количество нейронов в последнем слое должно соответствовать количеству классов, на которые вы классифицируете. В данном случае это цифры 0-9, поэтому их 10, поэтому в последнем слое должно быть 10 нейронов


In [ ]:
import tensorflow as tf
print(tf.__version__)

mnist = tf.keras.datasets.fashion_mnist

(training_images, training_labels) ,  (test_images, test_labels) = mnist.load_data()

training_images = training_images/255.0
test_images = test_images/255.0

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(64, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(5, activation=tf.nn.softmax)])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy')

model.fit(training_images, training_labels, epochs=5)

model.evaluate(test_images, test_labels)

classifications = model.predict(test_images)

print(classifications[0])
print(test_labels[0])

##Упражнение 5:

Рассмотрим влияние дополнительных слоев в сети. Что произойдет, если вы добавите еще один слой между слоем с 512 и последним слоем с 10.

Ответ: Это не оказывает существенного влияния, потому что это относительно простые данные. Для гораздо более сложных данных (включая цветные изображения, на которых нужно классифицировать цвета), часто необходимы дополнительные слои.

In [ ]:
import tensorflow as tf
print(tf.__version__)

mnist = tf.keras.datasets.fashion_mnist

(training_images, training_labels) ,  (test_images, test_labels) = mnist.load_data()

training_images = training_images/255.0
test_images = test_images/255.0

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(256, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy')

model.fit(training_images, training_labels, epochs=5)

model.evaluate(test_images, test_labels)

classifications = model.predict(test_images)

print(classifications[0])
print(test_labels[0])

##Упражнение 6:
Рассмотрим влияние обучения для большего или меньшего числа эпох. Как вы думаете, почему это так?

Попробуйте 15 эпох - вы, вероятно, получите модель с гораздо лучшим Loss, чем модель с 5
Попробуйте 30 эпох - вы можете увидеть, что значение потерь перестает уменьшаться, а иногда и увеличивается.
Более реальную картину процесса обучения нейронной сети можно увидеть, применив кросс-валидацию: на каждой эпохе модель будет проверяться на тестовом наборе данных, который не участвовал в обучении. Действительно, ведь сеть должна будет предсказывать категории картинок, которые раньше не видела, и ее производительность должна оцениваться на неизвестных данных. Тестовый набор заранее отбирается из большого обучаемого набора и тоже уже имеет метки к каждому примеру. Так мы сможем сравнивать реальные категории с предсказанными и вычислять реальные потери модели на данных, которые не участвовали в настройке весов.

При большом количестве эпох часто случается, что потери на обучающем наборе снижаются, а на тестовом наборе в какой-то момент начинают расти. Это означает, что модель хорошо выучила те признаки категорий, которые есть в обучающем наборе, но не оказалось в тестовых данных. То есть, выучила хорошо, но  признаки слишком специфичные, не общие для изображений этой категории. Так бывает, когда обучающий набор не содержит более обобщенных признаков в количестве, достаточном, чтобы они стали существенными признаками для модели, и она не присвоила им достаточно высокие веса. Это эффект, называемый «overfitting» или переобучение, за которым нужно следить при обучении нейронных сетей. Нет смысла тратить время на обучение сети, если потери на тестовых данных не уменьшаются.


In [ ]:
import tensorflow as tf
print(tf.__version__)

mnist = tf.keras.datasets.fashion_mnist

(training_images, training_labels) ,  (test_images, test_labels) = mnist.load_data()

training_images = training_images/255.0
test_images = test_images/255.0

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy')

model.fit(training_images, training_labels, validation_data=(test_images, test_labels), epochs=30)

model.evaluate(test_images, test_labels)

classifications = model.predict(test_images)

print(classifications[34])
print(test_labels[34])

##Упражнение 7:

Перед тренировкой вы нормализовали данные, перейдя от значений 0-255 к значениям 0-1. Как повлияет удаление нормализации? Вот полный код, чтобы попробовать. Как вы думаете, почему получаются разные результаты? Ответьте несколькими предложениями в лекции.

In [ ]:
import tensorflow as tf
print(tf.__version__)

mnist = tf.keras.datasets.fashion_mnist

(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
# training_images=training_images/255.0
# test_images=test_images/255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.fit(training_images, training_labels, epochs=5)
model.evaluate(test_images, test_labels)
classifications = model.predict(test_images)
print(classifications[0])
print(test_labels[0])

##Упражнение 8:
Ранее, во время тренировки с дополнительными эпохами, возникла проблема, из-за которой значение Loss могло ухудшиться. Возможно, это заняло некоторое лишнее время, и вы, возможно, подумали: «Разве не было бы хорошо, если бы я мог остановить обучение, когда достигну желаемого значения?» - то есть 95% точности может быть достаточно для вас, и если вы достигнете этого после 3 эпох, зачем сидеть сложа руки в ожидании, пока завершится большое количество эпох.
Так как бы вы это исправили? Как и в любой другой программе - у вас есть callback! Посмотрим на них в действии:

In [ ]:
import tensorflow as tf
print(tf.__version__)

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_loss')<0.34):
      print("\nValidation Loss reached 0.34,  so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()
mnist = tf.keras.datasets.fashion_mnist

(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images/255.0
test_images=test_images/255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.fit(training_images, training_labels, validation_data=(test_images, test_labels), epochs=10, callbacks=[callbacks])


